In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import lightgbm  # For early_stopping callback
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV
import warnings
import gc
warnings.filterwarnings('ignore')

# Define file paths
train_path = "/Users/saitejasuppu/Downloads/sp-25-sales-forecasting/train.csv"
test_path = "/Users/saitejasuppu/Downloads/sp-25-sales-forecasting/test.csv"
economic_indicators_path = "/Users/saitejasuppu/Downloads/sp-25-sales-forecasting/EconomicIndicators.csv"

# Load datasets
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
economic_data = pd.read_csv(economic_indicators_path)

# Handle missing values
train_data['InventoryRatio'].fillna(train_data['InventoryRatio'].median(), inplace=True)
test_data['InventoryRatio'].fillna(test_data['InventoryRatio'].median(), inplace=True)

# Convert Quarter to datetime
def convert_quarter_to_datetime(df, quarter_col='Quarter', base_year=2020):
    df['QuarterNum'] = df[quarter_col].str.extract(r'Q(\d+)').astype(int)
    df['Year'] = base_year + (df['QuarterNum'] - 1) // 4
    df['Month'] = (3 * ((df['QuarterNum'] - 1) % 4)) + 1
    df[quarter_col] = pd.to_datetime(df[['Year', 'Month']].assign(DAY=1))
    df.drop(columns=['QuarterNum', 'Year', 'Month'], inplace=True)
    return df

train_data = convert_quarter_to_datetime(train_data)
test_data = convert_quarter_to_datetime(test_data)

# Convert Economic Data Month to Quarter
economic_data['Year'] = 2020 + (economic_data['Month'] - 1) // 12
economic_data['Month'] = (economic_data['Month'] - 1) % 12 + 1
economic_data['Date'] = pd.to_datetime(economic_data[['Year', 'Month']].assign(DAY=1))
economic_data['Quarter'] = economic_data['Date'].dt.to_period('Q').astype(str)
economic_quarterly = economic_data.groupby('Quarter').mean().reset_index()
economic_quarterly['Year'] = economic_quarterly['Quarter'].str[:4].astype(int)
economic_quarterly['Q'] = economic_quarterly['Quarter'].str[-1].astype(int)
quarter_to_month = {1: 1, 2: 4, 3: 7, 4: 10}
economic_quarterly['Month'] = economic_quarterly['Q'].map(quarter_to_month)
economic_quarterly['Quarter'] = pd.to_datetime(economic_quarterly[['Year', 'Month']].assign(DAY=1))
economic_quarterly.drop(columns=['Year', 'Q', 'Month', 'Date'], inplace=True, errors='ignore')

# Merge with economic indicators
train_data = train_data.merge(economic_quarterly, on='Quarter', how='left')
test_data = test_data.merge(economic_quarterly, on='Quarter', how='left')

# Feature Engineering
# Lag Features: Previous Quarter Sales (1, 2, 3 quarters)
train_data = train_data.sort_values(['Company', 'Quarter'])
train_data['Prev_Quarter_Sales'] = train_data.groupby('Company')['Sales'].shift(1)
train_data['Prev_2_Quarter_Sales'] = train_data.groupby('Company')['Sales'].shift(2)
train_data['Prev_3_Quarter_Sales'] = train_data.groupby('Company')['Sales'].shift(3)
train_data['Prev_Quarter_Sales'].fillna(train_data['Sales'].median(), inplace=True)
train_data['Prev_2_Quarter_Sales'].fillna(train_data['Sales'].median(), inplace=True)
train_data['Prev_3_Quarter_Sales'].fillna(train_data['Sales'].median(), inplace=True)
latest_sales = train_data.groupby('Company')[['Sales']].last().reset_index()
latest_sales.rename(columns={'Sales': 'Prev_Quarter_Sales'}, inplace=True)
latest_sales_2 = train_data.groupby('Company')['Sales'].shift(1).groupby(train_data['Company']).last().reset_index()
latest_sales_2.rename(columns={'Sales': 'Prev_2_Quarter_Sales'}, inplace=True)
latest_sales_3 = train_data.groupby('Company')['Sales'].shift(2).groupby(train_data['Company']).last().reset_index()
latest_sales_3.rename(columns={'Sales': 'Prev_3_Quarter_Sales'}, inplace=True)
test_data = test_data.merge(latest_sales, on='Company', how='left')
test_data = test_data.merge(latest_sales_2, on='Company', how='left')
test_data = test_data.merge(latest_sales_3, on='Company', how='left')
test_data['Prev_Quarter_Sales'].fillna(train_data['Sales'].median(), inplace=True)
test_data['Prev_2_Quarter_Sales'].fillna(train_data['Sales'].median(), inplace=True)
test_data['Prev_3_Quarter_Sales'].fillna(train_data['Sales'].median(), inplace=True)

# Sales Trend Feature
train_data['Sales_Trend'] = train_data['Prev_Quarter_Sales'] - train_data['Prev_2_Quarter_Sales']
test_data['Sales_Trend'] = test_data['Prev_Quarter_Sales'] - test_data['Prev_2_Quarter_Sales']
train_data['Sales_Trend'].fillna(0, inplace=True)
test_data['Sales_Trend'].fillna(0, inplace=True)

# New Interaction Features
train_data['Sales_Trend_PMI'] = train_data['Sales_Trend'] * train_data['PMI']
test_data['Sales_Trend_PMI'] = test_data['Sales_Trend'] * test_data['PMI']
train_data['Sales_Trend_Interest'] = train_data['Sales_Trend'] * train_data['Interest Rate']
test_data['Sales_Trend_Interest'] = test_data['Sales_Trend'] * test_data['Interest Rate']

# Rolling Statistics for Sales
train_data['Rolling_Avg_Sales'] = train_data.groupby('Company')['Sales'].transform(lambda x: x.rolling(3, min_periods=1).mean())
train_data['Rolling_Std_Sales'] = train_data.groupby('Company')['Sales'].transform(lambda x: x.rolling(3, min_periods=1).std())
test_data = test_data.merge(train_data.groupby('Company')[['Rolling_Avg_Sales']].last().reset_index(), on='Company', how='left')
test_data = test_data.merge(train_data.groupby('Company')[['Rolling_Std_Sales']].last().reset_index(), on='Company', how='left')
train_data['Rolling_Std_Sales'].fillna(train_data['Rolling_Std_Sales'].median(), inplace=True)
test_data['Rolling_Avg_Sales'].fillna(train_data['Rolling_Avg_Sales'].median(), inplace=True)
test_data['Rolling_Std_Sales'].fillna(train_data['Rolling_Std_Sales'].median(), inplace=True)

# Industry-Level Average Sales (using past data only)
train_data = train_data.sort_values('Quarter')
industry_avg_sales = train_data.groupby(['Industry', 'Quarter'])['Sales'].transform(lambda x: x.shift(1).rolling(2, min_periods=1).mean())
train_data['Industry_Avg_Sales'] = industry_avg_sales
train_data['Industry_Avg_Sales'].fillna(train_data['Industry_Avg_Sales'].median(), inplace=True)
test_data = test_data.merge(train_data.groupby(['Industry', 'Quarter'])[['Industry_Avg_Sales']].last().reset_index(), on=['Industry', 'Quarter'], how='left')
test_data['Industry_Avg_Sales'].fillna(train_data['Industry_Avg_Sales'].median(), inplace=True)

# Historical Quarterly Sales Average (Seasonality)
train_data['Quarter_Num'] = train_data['Quarter'].dt.quarter
test_data['Quarter_Num'] = test_data['Quarter'].dt.quarter
quarterly_avg_sales = train_data.groupby('Quarter_Num')['Sales'].mean().to_dict()
train_data['Quarterly_Avg_Sales'] = train_data['Quarter_Num'].map(quarterly_avg_sales)
test_data['Quarterly_Avg_Sales'] = test_data['Quarter_Num'].map(quarterly_avg_sales)

# Interaction Features
train_data['Rev_to_Inventory'] = train_data['RevenueGrowth'] / (train_data['InventoryRatio'] + 1e-5)
test_data['Rev_to_Inventory'] = test_data['RevenueGrowth'] / (test_data['InventoryRatio'] + 1e-5)
train_data['Market_Effectiveness'] = train_data['Marketshare'] / (train_data['RevenueGrowth'] + 1e-5)
test_data['Market_Effectiveness'] = test_data['Marketshare'] / (test_data['RevenueGrowth'] + 1e-5)
train_data['PMI_Rev_Interaction'] = train_data['PMI'] * train_data['RevenueGrowth']
test_data['PMI_Rev_Interaction'] = test_data['PMI'] * test_data['RevenueGrowth']
train_data['PMI_Inventory_Interaction'] = train_data['PMI'] * train_data['InventoryRatio']
test_data['PMI_Inventory_Interaction'] = test_data['PMI'] * test_data['InventoryRatio']
train_data['Interest_Market_Interaction'] = train_data['Interest Rate'] * train_data['Marketshare']
test_data['Interest_Market_Interaction'] = test_data['Interest Rate'] * test_data['Marketshare']
train_data['PMI_Prev_Sales'] = train_data['PMI'] * train_data['Prev_Quarter_Sales']
test_data['PMI_Prev_Sales'] = test_data['PMI'] * test_data['Prev_Quarter_Sales']
train_data['Interest_Prev_Sales'] = train_data['Interest Rate'] * train_data['Prev_Quarter_Sales']
test_data['Interest_Prev_Sales'] = test_data['Interest Rate'] * test_data['Prev_Quarter_Sales']

# Cyclical Encoding for Quarter
train_data['Year'] = train_data['Quarter'].dt.year
test_data['Year'] = test_data['Quarter'].dt.year
train_data['Quarter_Sin'] = np.sin(2 * np.pi * train_data['Quarter_Num'] / 4)
train_data['Quarter_Cos'] = np.cos(2 * np.pi * train_data['Quarter_Num'] / 4)
test_data['Quarter_Sin'] = np.sin(2 * np.pi * test_data['Quarter_Num'] / 4)
test_data['Quarter_Cos'] = np.cos(2 * np.pi * test_data['Quarter_Num'] / 4)

# Cap Outliers in InventoryRatio
train_data['InventoryRatio'] = train_data['InventoryRatio'].clip(upper=train_data['InventoryRatio'].quantile(0.95))
test_data['InventoryRatio'] = test_data['InventoryRatio'].clip(upper=test_data['InventoryRatio'].quantile(0.95))

# Lagged Economic Indicators
train_data['PMI_Lag1'] = train_data['PMI'].shift(1)
train_data['Interest_Rate_Lag1'] = train_data['Interest Rate'].shift(1)
train_data['PMI_Lag1'].fillna(train_data['PMI'].median(), inplace=True)
train_data['Interest_Rate_Lag1'].fillna(train_data['Interest Rate'].median(), inplace=True)
test_data['PMI_Lag1'] = train_data['PMI'].iloc[-1]  # Use the last available PMI
test_data['Interest_Rate_Lag1'] = train_data['Interest Rate'].iloc[-1]  # Use the last available Interest Rate

# Difference Features
train_data['Marketshare_Diff'] = train_data.groupby('Company')['Marketshare'].diff().fillna(0)
train_data['RevenueGrowth_Diff'] = train_data.groupby('Company')['RevenueGrowth'].diff().fillna(0)
test_data['Marketshare_Diff'] = test_data['Marketshare'] - train_data.groupby('Company')['Marketshare'].last().reindex(test_data['Company']).values
test_data['RevenueGrowth_Diff'] = test_data['RevenueGrowth'] - train_data.groupby('Company')['RevenueGrowth'].last().reindex(test_data['Company']).values
test_data['Marketshare_Diff'].fillna(0, inplace=True)
test_data['RevenueGrowth_Diff'].fillna(0, inplace=True)

# Target Encoding for Company and Industry
def target_encode(df_train, df_test, column, target, alpha=5):
    global_mean = df_train[target].mean()
    grouped = df_train.groupby(column)[target].agg(['mean', 'count'])
    grouped['smooth_mean'] = (grouped['mean'] * grouped['count'] + global_mean * alpha) / (grouped['count'] + alpha)
    df_train[f'{column}_Target_Encoded'] = df_train[column].map(grouped['smooth_mean'])
    df_test[f'{column}_Target_Encoded'] = df_test[column].map(grouped['smooth_mean']).fillna(global_mean)
    return df_train, df_test

train_data, test_data = target_encode(train_data, test_data, 'Company', 'Sales')
train_data, test_data = target_encode(train_data, test_data, 'Industry', 'Sales')

# Encode Categorical Features
categorical_columns = ['Company', 'Bond rating', 'Stock rating', 'Region', 'Industry']
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = test_data[col].apply(lambda x: le.transform([x])[0] if x in le.classes_ else -1)
    label_encoders[col] = le

# Drop Quarter column from both train and test data
train_data.drop(columns=['Quarter'], inplace=True)
test_data.drop(columns=['Quarter'], inplace=True)

# Define Features
features = ['Company', 'QuickRatio', 'InventoryRatio', 'RevenueGrowth', 'Marketshare',
            'Bond rating', 'Stock rating', 'Region', 'Industry', 'Year', 'Quarter_Num',
            'Rev_to_Inventory', 'Market_Effectiveness', 'Industry_Avg_Sales',
            'Prev_Quarter_Sales', 'Prev_2_Quarter_Sales', 'Prev_3_Quarter_Sales',
            'Rolling_Avg_Sales', 'Rolling_Std_Sales', 'PMI_Rev_Interaction',
            'PMI_Inventory_Interaction', 'Interest_Market_Interaction',
            'Quarter_Sin', 'Quarter_Cos', 'Consumer Sentiment', 'Interest Rate', 'PMI',
            'Money Supply', 'NationalEAI', 'EastEAI', 'WestEAI', 'SouthEAI', 'NorthEAI',
            'Sales_Trend', 'PMI_Prev_Sales', 'Interest_Prev_Sales',
            'Sales_Trend_PMI', 'Sales_Trend_Interest', 'Quarterly_Avg_Sales',
            'PMI_Lag1', 'Interest_Rate_Lag1', 'Marketshare_Diff', 'RevenueGrowth_Diff',
            'Company_Target_Encoded', 'Industry_Target_Encoded']

# Prepare Training and Validation Sets (Use Most Recent 20% for Validation)
train_data = train_data.sort_values(['Company', 'Year', 'Quarter_Num'])
n = len(train_data)
train_size = int(0.8 * n)
X = train_data[features]
y = train_data['Sales']
X_train = X.iloc[:train_size]
y_train = y.iloc[:train_size]
X_val = X.iloc[train_size:]
y_val = y.iloc[train_size:]

# Clean up memory
gc.collect()

# Model 1: XGBoost with Enhanced Hyperparameters
xgb_params = {
    'n_estimators': [200, 300, 400],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.03],
    'subsample': [0.6, 0.7, 0.8],  # Reduced to add dropout
    'colsample_bytree': [0.6, 0.7, 0.8],  # Reduced to add dropout
    'reg_lambda': [30, 50],  # Increased regularization
    'reg_alpha': [15, 20],  # Increased regularization
    'min_child_weight': [15, 20]  # Increased to control overfitting
}
xgb_model = XGBRegressor(random_state=42, early_stopping_rounds=50)  # Increased early stopping rounds
xgb_search = RandomizedSearchCV(xgb_model, xgb_params, n_iter=20, cv=TimeSeriesSplit(n_splits=5), scoring='neg_mean_absolute_error', random_state=42)
xgb_search.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
best_xgb_params = xgb_search.best_params_
print(f"Best XGBoost Parameters: {best_xgb_params}")

# Create a new XGBoost model without early_stopping_rounds for stacking
best_xgb = XGBRegressor(**best_xgb_params, random_state=42)
best_xgb.fit(X_train, y_train)

# Compute Train and Validation MAE for XGBoost
y_train_pred_xgb = xgb_search.best_estimator_.predict(X_train)
y_val_pred_xgb = xgb_search.best_estimator_.predict(X_val)
mae_xgb_train = mean_absolute_error(y_train, y_train_pred_xgb)
mae_xgb_val = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"XGBoost Train MAE: {mae_xgb_train}")
print(f"XGBoost Validation MAE: {mae_xgb_val}")

# Feature Importance Analysis (Drop Low-Impact Features)
feature_importances = pd.DataFrame({
    'feature': features,
    'importance': best_xgb.feature_importances_
}).sort_values('importance', ascending=False)
print("\nFeature Importances:\n", feature_importances)

# Drop features with importance < 0.01
low_impact_features = feature_importances[feature_importances['importance'] < 0.01]['feature'].tolist()
print(f"\nDropping Low-Impact Features: {low_impact_features}")
features = [f for f in features if f not in low_impact_features]
X_train = X_train[features]
X_val = X_val[features]
test_data = test_data[features + ['RowID']]  # Keep RowID for submission

# Retrain XGBoost on Reduced Feature Set
best_xgb.fit(X_train, y_train)
y_train_pred_xgb = best_xgb.predict(X_train)
y_val_pred_xgb = best_xgb.predict(X_val)
mae_xgb_train = mean_absolute_error(y_train, y_train_pred_xgb)
mae_xgb_val = mean_absolute_error(y_val, y_val_pred_xgb)
print(f"XGBoost Train MAE (After Feature Selection): {mae_xgb_train}")
print(f"XGBoost Validation MAE (After Feature Selection): {mae_xgb_val}")

# Clean up memory
del xgb_model, xgb_search
gc.collect()

# Model 2: LightGBM with Optimized Parameters
lgbm_params = {
    'n_estimators': [200, 300, 400],
    'max_depth': [5, 7],  # Increased for more flexibility
    'learning_rate': [0.01, 0.03],
    'num_leaves': [15, 31, 50],  # Increased to capture more patterns
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'min_data_in_leaf': [10, 20],  # Reduced to allow more splits
    'lambda_l1': [0.1, 0.5],
    'lambda_l2': [0.1, 0.5]
}
lgbm_model = LGBMRegressor(random_state=42)
lgbm_search = RandomizedSearchCV(lgbm_model, lgbm_params, n_iter=20, cv=TimeSeriesSplit(n_splits=5), scoring='neg_mean_absolute_error', random_state=42)
lgbm_search.fit(X_train, y_train)
best_params = lgbm_search.best_params_
print(f"Best LightGBM Parameters: {best_params}")

best_lgbm = LGBMRegressor(**best_params, random_state=42)
best_lgbm.fit(
    X_train, y_train,
    eval_metric='mae',
    eval_set=[(X_val, y_val)],
    callbacks=[lightgbm.early_stopping(stopping_rounds=20, verbose=False)]
)

y_train_pred_lgbm = best_lgbm.predict(X_train)
y_val_pred_lgbm = best_lgbm.predict(X_val)
mae_lgbm_train = mean_absolute_error(y_train, y_train_pred_lgbm)
mae_lgbm_val = mean_absolute_error(y_val, y_val_pred_lgbm)
print(f"LightGBM Train MAE (After Feature Selection): {mae_lgbm_train}")
print(f"LightGBM Validation MAE (After Feature Selection): {mae_lgbm_val}")

# Clean up memory
del lgbm_model, lgbm_search
gc.collect()

# Model 3: CatBoost
catboost_params = {
    'iterations': [200, 300, 400],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.03],
    'l2_leaf_reg': [3, 5, 7],
    'subsample': [0.7, 0.8, 0.9]
}
catboost_model = CatBoostRegressor(random_state=42, verbose=0)
catboost_search = RandomizedSearchCV(catboost_model, catboost_params, n_iter=20, cv=TimeSeriesSplit(n_splits=5), scoring='neg_mean_absolute_error', random_state=42)
catboost_search.fit(X_train, y_train)
best_catboost = catboost_search.best_estimator_

best_catboost.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=20, verbose=0)

y_train_pred_catboost = best_catboost.predict(X_train)
y_val_pred_catboost = best_catboost.predict(X_val)
mae_catboost_train = mean_absolute_error(y_train, y_train_pred_catboost)
mae_catboost_val = mean_absolute_error(y_val, y_val_pred_catboost)
print(f"CatBoost Train MAE (After Feature Selection): {mae_catboost_train}")
print(f"CatBoost Validation MAE (After Feature Selection): {mae_catboost_val}")

# Clean up memory
del catboost_model, catboost_search
gc.collect()

# Stacking Ensemble
estimators = [
    ('xgb', best_xgb),
    ('lgbm', best_lgbm),
    ('catboost', best_catboost)
]
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)
stacking_model.fit(X_train, y_train)

y_train_pred_stack = stacking_model.predict(X_train)
y_val_pred_stack = stacking_model.predict(X_val)
mae_stack_train = mean_absolute_error(y_train, y_train_pred_stack)
mae_stack_val = mean_absolute_error(y_val, y_val_pred_stack)
print(f"Stacking Ensemble Train MAE (After Feature Selection): {mae_stack_train}")
print(f"Stacking Ensemble Validation MAE (After Feature Selection): {mae_stack_val}")

# Time-Series Cross-Validation for Stacking Model
tscv = TimeSeriesSplit(n_splits=5)
cv_maes = []
for train_idx, val_idx in tscv.split(X):
    X_tr, X_v = X.iloc[train_idx][features], X.iloc[val_idx][features]
    y_tr, y_v = y.iloc[train_idx], y.iloc[val_idx]
    stacking_model.fit(X_tr, y_tr)
    y_v_pred = stacking_model.predict(X_v)
    cv_maes.append(mean_absolute_error(y_v, y_v_pred))
print(f"Stacking Ensemble Time-Series CV MAE: {np.mean(cv_maes)} ± {np.std(cv_maes)}")

# Clean up memory
gc.collect()

# Optimize Ensemble Weights Using Time-Series Cross-Validation
weight_combinations = [
    (w_xgb, w_lgbm, w_catboost, w_stack)
    for w_xgb in [0.3, 0.35, 0.4, 0.45]
    for w_lgbm in [0.2, 0.25, 0.3]
    for w_catboost in [0.1, 0.15, 0.2]
    for w_stack in [0.1, 0.15, 0.2]
    if abs(w_xgb + w_lgbm + w_catboost + w_stack - 1.0) < 1e-5
]

best_mae = float('inf')
best_weights = None
tscv = TimeSeriesSplit(n_splits=5)
for weights in weight_combinations:
    w_xgb, w_lgbm, w_catboost, w_stack = weights
    cv_maes = []
    for train_idx, val_idx in tscv.split(X):
        X_tr, X_v = X.iloc[train_idx][features], X.iloc[val_idx][features]
        y_tr, y_v = y.iloc[train_idx], y.iloc[val_idx]
        y_tr_pred_xgb = best_xgb.predict(X_tr)
        y_v_pred_xgb = best_xgb.predict(X_v)
        y_tr_pred_lgbm = best_lgbm.predict(X_tr)
        y_v_pred_lgbm = best_lgbm.predict(X_v)
        y_tr_pred_catboost = best_catboost.predict(X_tr)
        y_v_pred_catboost = best_catboost.predict(X_v)
        y_tr_pred_stack = stacking_model.predict(X_tr)
        y_v_pred_stack = stacking_model.predict(X_v)
        ensemble_pred = (w_xgb * y_v_pred_xgb + 
                         w_lgbm * y_v_pred_lgbm + 
                         w_catboost * y_v_pred_catboost + 
                         w_stack * y_v_pred_stack)
        mae = mean_absolute_error(y_v, ensemble_pred)
        cv_maes.append(mae)
    mean_mae = np.mean(cv_maes)
    if mean_mae < best_mae:
        best_mae = mean_mae
        best_weights = {'xgb': w_xgb, 'lgbm': w_lgbm, 'catboost': w_catboost, 'stack': w_stack}

print(f"Optimized Ensemble Weights (via Time-Series CV): {best_weights}")
print(f"Ensemble Time-Series CV MAE with Optimized Weights: {best_mae}")

# Final Predictions with Optimized Weights
test_pred_xgb = best_xgb.predict(test_data[features])
test_pred_lgbm = best_lgbm.predict(test_data[features])
test_pred_catboost = best_catboost.predict(test_data[features])
test_pred_stack = stacking_model.predict(test_data[features])

final_predictions = (
    best_weights['xgb'] * test_pred_xgb +
    best_weights['lgbm'] * test_pred_lgbm +
    best_weights['catboost'] * test_pred_catboost +
    best_weights['stack'] * test_pred_stack
)

# Post-Processing: Clip Predictions to Training Sales Range
sales_lower_bound = train_data['Sales'].quantile(0.01)
sales_upper_bound = train_data['Sales'].quantile(0.99)
final_predictions = np.clip(final_predictions, sales_lower_bound, sales_upper_bound)

# Save Submission
submission = pd.DataFrame({'RowID': test_data['RowID'], 'Sales': final_predictions})
submission['Sales'] = submission['Sales'].round().astype(int)
submission.to_csv("/Users/saitejasuppu/Downloads/submission_stacking_optimized_v11.csv", index=False)
print("✅ Predictions saved as submission_stacking_optimized_v11.csv")

# XGBoost-Only Submission
xgb_predictions = best_xgb.predict(test_data[features])
xgb_predictions = np.clip(xgb_predictions, sales_lower_bound, sales_upper_bound)
submission_xgb = pd.DataFrame({'RowID': test_data['RowID'], 'Sales': xgb_predictions})
submission_xgb['Sales'] = submission_xgb['Sales'].round().astype(int)
submission_xgb.to_csv("/Users/saitejasuppu/Downloads/submission_xgboost_optimized_v11.csv", index=False)
print("✅ XGBoost predictions saved as submission_xgboost_optimized_v11.csv")

Best XGBoost Parameters: {'subsample': 0.7, 'reg_lambda': 30, 'reg_alpha': 15, 'n_estimators': 400, 'min_child_weight': 15, 'max_depth': 3, 'learning_rate': 0.03, 'colsample_bytree': 0.8}
XGBoost Train MAE: 272.14496343703496
XGBoost Validation MAE: 415.6189860026042

Feature Importances:
                         feature  importance
17            Rolling_Avg_Sales    0.273752
43       Company_Target_Encoded    0.255986
18            Rolling_Std_Sales    0.037963
2                InventoryRatio    0.036488
20    PMI_Inventory_Interaction    0.030984
14           Prev_Quarter_Sales    0.023604
34               PMI_Prev_Sales    0.021978
15         Prev_2_Quarter_Sales    0.019279
28                  NationalEAI    0.019240
16         Prev_3_Quarter_Sales    0.018893
23                  Quarter_Cos    0.017780
37         Sales_Trend_Interest    0.016050
33                  Sales_Trend    0.015820
36              Sales_Trend_PMI    0.014981
1                    QuickRatio    0.013851
35   

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.5, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.5
[LightGBM] [Warning] min_data_in_leaf is set=20, min_ch

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [W

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [W

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [W

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [W

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
[LightGBM] [Warning] lambda_l2 is set=0.1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [W